# 用 pytorch 导出一个复杂一点的模型进行 子模型 提取

In [1]:
import torch

class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.convs1 = torch.nn.Sequential(torch.nn.Conv2d(3, 3, 3),
                                          torch.nn.Conv2d(3, 3, 3),
                                          torch.nn.Conv2d(3, 3, 3))
        self.convs2 = torch.nn.Sequential(torch.nn.Conv2d(3, 3, 3),
                                          torch.nn.Conv2d(3, 3, 3))
        self.convs3 = torch.nn.Sequential(torch.nn.Conv2d(3, 3, 3),
                                          torch.nn.Conv2d(3, 3, 3))
        self.convs4 = torch.nn.Sequential(torch.nn.Conv2d(3, 3, 3),
                                          torch.nn.Conv2d(3, 3, 3),
                                          torch.nn.Conv2d(3, 3, 3))
    def forward(self, x):
        x = self.convs1(x)
        x1 = self.convs2(x)
        x2 = self.convs3(x)
        x = x1 + x2
        x = self.convs4(x)
        return x

model = Model()
input = torch.randn(1, 3, 20, 20)

torch.onnx.export(model, input, 'whole_model.onnx')

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



## 提取一个子模型

In [12]:
import onnx
import onnx.utils

onnx.utils.extract_model('whole_model.onnx', 'partial_model.onnx', input_names = ['/convs1/convs1.1/Conv_output_0'], output_names=["/Add_output_0"])

## 添加额外输出

In [15]:
onnx.utils.extract_model('whole_model.onnx', 'submodel_1.onnx', ['/convs1/convs1.1/Conv_output_0'], ['/convs3/convs3.1/Conv_output_0', '31'])

## 添加冗余输入

In [16]:
onnx.utils.extract_model('whole_model.onnx', 'partial_model.onnx', input_names = ['/convs1/convs1.1/Conv_output_0', 'input.1'], output_names=["/Add_output_0"])

## 提取双边输入的 子模型

In [17]:
onnx.utils.extract_model('whole_model.onnx', 'new.onnx', input_names=['/convs2/convs2.0/Conv_output_0', "/convs3/convs3.0/Conv_output_0"], output_names=["/Add_output_0"])

## 输出中间节点的 一些输出值

In [18]:
onnx.utils.extract_model('whole_model.onnx', 'more_output_model.onnx', ['input.1'], ['31', "/convs1/convs1.2/Conv_output_0", "/convs2/convs2.1/Conv_output_0", "/convs3/convs3.1/Conv_output_0"])

## 拆分之前 的 `more_output_model.onnx` 为多个子模型，方便对部分模块进行调试

In [19]:
onnx.utils.extract_model('whole_model.onnx', 'debug_model_1.onnx', ['input.1'], ["/convs1/convs1.2/Conv_output_0"])
onnx.utils.extract_model('whole_model.onnx', 'debug_model_2.onnx', ["/convs1/convs1.2/Conv_output_0"], ["/convs2/convs2.1/Conv_output_0"])

# 好麻烦，不想找了，前面两个展示了 
# onnx.utils.extract_model('whole_model.onnx', 'debug_model_3.onnx', ['23'], ['27'])
# onnx.utils.extract_model('whole_model.onnx', 'debug_model_4.onnx', ['25', '27'], ['31'])